In [1]:
import json
import pandas as pd

# 1. JSON 데이터 (예시)
with open("../public/data.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)


# 2. DataFrame 변환
df = pd.DataFrame(json_data)

# 3. df1: 순공시간 - 날짜
df1 = df[["날짜", "순공시간"]]

# 4. df2: 순공시간_수행평가 - 날짜
df2 = df[["날짜", "순공시간_수행평가"]]

# 5. df3: 과목별 플랜 수 - 날짜
df3 = df.groupby(["날짜", "과목"]).size().reset_index(name="플랜수")

# 6. HTML 테이블로 저장
with open("table1.html", "w", encoding="utf-8") as f:
    f.write("<h2>df1: 순공시간 - 날짜</h2>")
    f.write(df1.to_html(index=False))
    f.write("<h2>df2: 순공시간_수행평가 - 날짜</h2>")
    f.write(df2.to_html(index=False))
    f.write("<h2>df3: 과목별 플랜수 - 날짜</h2>")
    f.write(df3.to_html(index=False))

In [3]:
import json
import pandas as pd

# 📥 JSON 불러오기 (상대경로)
with open("../public/data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df["날짜"] = pd.to_datetime(df["날짜"])
df["날짜"] = df["날짜"].dt.strftime("%Y-%m-%d")

# NaN을 0으로 처리
df["순공시간"] = df["순공시간"].fillna(0)
df["순공시간_수행평가"] = df["순공시간_수행평가"].fillna(0)

# 1️⃣ 순공시간 총합
df1 = df.groupby("날짜")["순공시간"].sum().reset_index()

# 2️⃣ 수행평가 시간 총합 (NaN을 0으로 대체 후 처리)
df2 = df.groupby("날짜")["순공시간_수행평가"].sum().reset_index()

# 3️⃣ 날짜+과목별 플랜 수
df3 = df.groupby(["날짜", "과목"]).size().reset_index(name="플랜수")

# 📤 저장
df1.to_json("../public/study_time.json", orient="records", force_ascii=False)
df2.to_json("../public/eval_time.json", orient="records", force_ascii=False)
df3.to_json("../public/plan_count.json", orient="records", force_ascii=False)


In [8]:
# JSON → DataFrame
import json
import pandas as pd

with open("../public/data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df["날짜"] = pd.to_datetime(df["날짜"])
df["날짜"] = df["날짜"].dt.strftime("%Y-%m-%d")

# NaN을 0으로 채우기
df["순공시간"] = df["순공시간"].fillna(0)
df["순공시간_수행평가"] = df["순공시간_수행평가"].fillna(0)

# 날짜별 플랜 수
df3 = df.groupby(["날짜", "과목"]).size().reset_index(name="플랜수")

# ➕ 피벗 테이블로 변환 (과목별 컬럼으로)
pivot_df = df3.pivot(index="날짜", columns="과목", values="플랜수").fillna(0).reset_index()

# 저장 (columns: 날짜, 수학, 영어, 과학 ...)
pivot_df.to_json("../public/plan_count.json", orient="records", force_ascii=False)


In [9]:
import json
import pandas as pd

# 📥 JSON 불러오기
with open("../public/data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)
df["날짜"] = pd.to_datetime(df["날짜"])
df["날짜"] = df["날짜"].dt.strftime("%Y-%m-%d")

# 비어있는 값은 0으로 처리
df["순공시간"] = df["순공시간"].fillna(0)
df["순공시간_수행평가"] = df["순공시간_수행평가"].fillna(0)

# 📅 전체 날짜 리스트
all_dates = pd.date_range(df["날짜"].min(), df["날짜"].max()).strftime("%Y-%m-%d")

# ✅ 1. 순공시간 (study_time.json)
df1 = df.groupby("날짜")["순공시간"].sum().reindex(all_dates, fill_value=0).reset_index()
df1.columns = ["날짜", "순공시간"]
df1.to_json("../public/study_time.json", orient="records", force_ascii=False)

# ✅ 2. 수행평가 시간 (eval_time.json)
df2 = df.groupby("날짜")["순공시간_수행평가"].sum().reindex(all_dates, fill_value=0).reset_index()
df2.columns = ["날짜", "순공시간_수행평가"]
df2.to_json("../public/eval_time.json", orient="records", force_ascii=False)

# ✅ 3. 과목별 플랜 수 (plan_count.json)
all_subjects = df["과목"].dropna().unique()
full_index = pd.MultiIndex.from_product([all_dates, all_subjects], names=["날짜", "과목"])
df3 = df.groupby(["날짜", "과목"]).size().reset_index(name="플랜수")
df3_full = pd.DataFrame(index=full_index).reset_index()
df3_merged = pd.merge(df3_full, df3, on=["날짜", "과목"], how="left").fillna(0)
df3_pivot = df3_merged.pivot(index="날짜", columns="과목", values="플랜수").fillna(0).reset_index()
df3_pivot.to_json("../public/plan_count.json", orient="records", force_ascii=False)


TypeError: '<=' not supported between instances of 'str' and 'float'

In [14]:
import json
import pandas as pd

# 📥 JSON 불러오기 (상대경로)
with open("../public/data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 📊 DataFrame 변환
df = pd.DataFrame(data)

# ⏰ 시간 관련 결측값 0으로 채우기
df["순공시간"] = df["순공시간"].fillna(0)
df["순공시간_수행평가"] = df["순공시간_수행평가"].fillna(0)

# 🧹 날짜/과목 결측 제거 또는 처리
df = df.dropna(subset=["날짜", "과목"])  # 누락된 날짜/과목 제거
# df["과목"] = df["과목"].fillna("기타")  # 만약 기타로 채우고 싶으면 위 줄 대신 이 줄

# 📅 날짜 형식 통일
ddf = df[df["날짜"].apply(lambda x: isinstance(x, str))]
df["날짜"] = pd.to_datetime(df["날짜"], errors="coerce")
df = df.dropna(subset=["날짜"])
df["날짜"] = df["날짜"].dt.strftime("%Y-%m-%d")

# 📆 전체 날짜 범위 생성
all_dates = pd.date_range(df["날짜"].min(), df["날짜"].max()).strftime("%Y-%m-%d")

# ✅ 1. 순공시간 (study_time.json)
df1 = df.groupby("날짜")["순공시간"].sum().reindex(all_dates, fill_value=0).reset_index()
df1.columns = ["날짜", "순공시간"]
df1.to_json("../public/study_time.json", orient="records", force_ascii=False)

# ✅ 2. 수행평가 순공시간 (eval_time.json)
df2 = df.groupby("날짜")["순공시간_수행평가"].sum().reindex(all_dates, fill_value=0).reset_index()
df2.columns = ["날짜", "순공시간_수행평가"]
df2.to_json("../public/eval_time.json", orient="records", force_ascii=False)

# ✅ 3. 과목별 플랜 수 (plan_count.json)
df3 = df.groupby(["날짜", "과목"]).size().reset_index(name="플랜수")

# 📌 과목 전체 목록 확보
all_subjects = df["과목"].unique()

# 📌 날짜 × 과목 조합 MultiIndex로 생성
full_index = pd.MultiIndex.from_product([all_dates, all_subjects], names=["날짜", "과목"])
df3_full = pd.DataFrame(index=full_index).reset_index()

# 📌 기존 데이터 병합 및 누락 0 처리
df3_merged = pd.merge(df3_full, df3, on=["날짜", "과목"], how="left").fillna(0)

# 📌 피벗: 날짜별 과목별 컬럼화
df3_pivot = df3_merged.pivot(index="날짜", columns="과목", values="플랜수").fillna(0).reset_index()

# 🔍 혹시 모를 "" 열 제거
df3_pivot = df3_pivot.drop(columns=[""], errors="ignore")

# 📤 저장
df3_pivot.to_json("../public/plan_count.json", orient="records", force_ascii=False)


In [15]:
import json
import pandas as pd

# 📥 JSON 불러오기
with open("../public/data.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 📊 DataFrame으로 변환
df = pd.DataFrame(data)

# 🔍 유효한 과목 데이터만 추출
df = df.dropna(subset=["과목"])

# 📌 과목별 플랜 수 집계
df_subject_count = df["과목"].value_counts().reset_index()
df_subject_count.columns = ["과목", "플랜수"]

# 📤 JSON으로 저장
df_subject_count.to_json("../public/subject_plan_count.json", orient="records", force_ascii=False)
